# GeoTIFF to RST Converter for TerrSet
This notebook converts GeoTIFF files to RST format using parallel processing for speed.

In [ ]:
import os
import glob
from multiprocessing import Pool, cpu_count
from osgeo import gdal
import time

In [ ]:
def convert_tiff_to_rst(input_file):
    """Convert a single TIFF file to RST format"""
    try:
        output_file = input_file.replace('.tif', '.rst').replace('.tiff', '.rst')
        
        # Open source dataset
        src_ds = gdal.Open(input_file)
        if src_ds is None:
            print(f"Failed to open {input_file}")
            return False
            
        # Create RST file
        driver = gdal.GetDriverByName('RST')
        dst_ds = driver.CreateCopy(output_file, src_ds)
        
        # Clean up
        src_ds = None
        dst_ds = None
        
        print(f"Converted: {os.path.basename(input_file)} -> {os.path.basename(output_file)}")
        return True
        
    except Exception as e:
        print(f"Error converting {input_file}: {str(e)}")
        return False

In [ ]:
# Set your input folder path here
input_folder = "/home/gisuser/data/"  # Change this to your TIFF files location

# Find all TIFF files
tiff_patterns = [
    os.path.join(input_folder, "*.tif"),
    os.path.join(input_folder, "*.tiff"),
    os.path.join(input_folder, "**/*.tif"),
    os.path.join(input_folder, "**/*.tiff")
]

tiff_files = []
for pattern in tiff_patterns:
    tiff_files.extend(glob.glob(pattern, recursive=True))

print(f"Found {len(tiff_files)} TIFF files to convert")
for f in tiff_files[:5]:  # Show first 5 files
    print(f"  {f}")
if len(tiff_files) > 5:
    print(f"  ... and {len(tiff_files)-5} more")

In [ ]:
# Convert files using parallel processing
if tiff_files:
    num_processes = cpu_count()
    print(f"Using {num_processes} processes for conversion")
    
    start_time = time.time()
    
    with Pool(processes=num_processes) as pool:
        results = pool.map(convert_tiff_to_rst, tiff_files)
    
    end_time = time.time()
    successful = sum(results)
    
    print(f"\nConversion complete in {end_time - start_time:.2f} seconds")
    print(f"Successfully converted: {successful}/{len(tiff_files)} files")
else:
    print("No TIFF files found to convert")